In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import skfuzzy as fuzz
from scipy.stats import multivariate_normal 
from sklearn.preprocessing import StandardScaler

In [2]:
# Load and display each .nii file
loaded_images = []
for i, image_file in enumerate(image_files):
image_path = os.path.join(image_dir, image_file)
try:
# Load the NIfTI file
img = nib.load(image_path)
img_data = img.get_fdata() # Get the image data as a NumPy array
# Print the shape of the image data
print(f"Image {i+1}: {image_file}, shape: {img_data.shape}")
# Display the middle slice of the 3D image along the first axis
middle_slice = img_data[img_data.shape[0] // 2, :, :]
plt.figure()
plt.imshow(middle_slice.T, cmap="gray", origin="lower")
plt.title(f"Image {i+1}: {image_file} - Middle Slice")
plt.axis("off")
plt.show()
except Exception as e:
print(f"Error loading image {image_file}: {e}")


In [3]:
loaded_images = []
for i, image_file in enumerate(image_files):
image_path = os.path.join(image_dir, image_file)
try:
# Load the NIfTI file
img = nib.load(image_path)
img_data = img.get_fdata()

flattened_images = [img.flatten() for img in loaded_images]

In [4]:
n_clusters = 2
m = 2

In [5]:
cntr, u, u0,d,jm,p, fpc = fuzz.cluster.cmeans(
    flattened_images.T, n_clusters, m, error=0.005, maxiter=1000, init=None
)

In [6]:
cluster = np.argmax(u, axis=0)
for cluster in range(n_clusters):
cluster_size = np.sum(cluster_labels == cluster)
print(f"Cluster {cluster} contains {cluster_size} images.")

In [7]:
for cluster in range(n_cluster):
cluster_size = np.sum(cluster_labels == cluster)
if cluster_size > 0:
plt.figure()
cluster_images = np.array(loaded_images)[cluster_labels == cluster]
plt.imshow(cluster_images[0][..., cluster_images[0].shape[2] // 2], cmap="gray") # Middle slice
plt.title(f"Cluster {cluster} - Sample Image")
plt.axis("off")
plt.show()
else:
print(f"Cluster {cluster} is empty.")

In [8]:
def build_cnn_classifier(input_shape):
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax')) # 3 output classes (clusters)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
return model

generated_labels = tf.keras.utils.to_binary(generated_labels, num_classes=2)

cnn_model = build_cnn_classifier(target_shape)

cnn_model.fit(generated_images, generated_labels, epochs=10, batch_size=32)
